<a href="https://colab.research.google.com/github/gauravpalekar1980/Capstone_Projects/blob/main/FINAL_COPY_using_MSMARCO_EMBEDDING_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain_community
!pip install langchain-core
!pip install gradio
!pip install langchain_huggingface
!pip install huggingface_hub
!pip install datasets
!pip install chromadb
!pip install sentence-transformers
!pip install langchain-text-splitters
!pip install nltk
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 119.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
import os
import torch
from datasets import load_dataset

In [ ]:
from langchain.schema import Document

In [ ]:
#from huggingface_hub import login
#login()

### *** Load the RAGBENCH Train Data ***

In [ ]:


# load train/validation/test splits of individual subset
ragbench_covidqa = load_dataset("rungalileo/ragbench", "covidqa",split="test")
ragbench_hotpotqa = load_dataset("rungalileo/ragbench", "hotpotqa",split="test")
ragbench_cuad = load_dataset("rungalileo/ragbench", "cuad",split="test")
ragbench_delucionqa = load_dataset("rungalileo/ragbench", "delucionqa",split="test")
ragbench_emanual = load_dataset("rungalileo/ragbench", "emanual",split="test")
ragbench_expertqa = load_dataset("rungalileo/ragbench", "expertqa",split="test")
ragbench_finqa = load_dataset("rungalileo/ragbench", "finqa",split="test")
ragbench_hagrid = load_dataset("rungalileo/ragbench", "hagrid",split="test")
ragbench_msmarco = load_dataset("rungalileo/ragbench", "msmarco",split="test")
ragbench_pubmedqa = load_dataset("rungalileo/ragbench", "pubmedqa",split="test")
ragbench_tatqa = load_dataset("rungalileo/ragbench", "tatqa",split="test")
ragbench_techqa = load_dataset("rungalileo/ragbench", "techqa",split="test")


# load a specific split of a subset dataset
#ragbench_hotpotqa = load_dataset("rungalileo/ragbench", "hotpotqa", split="test")

# load the full ragbench dataset
#ragbench = {}
#for dataset in ['covidqa', 'hotpotqa']:#,'cuad', 'delucionqa', 'emanual', 'expertqa', 'finqa', 'hagrid', 'msmarco', 'pubmedqa', 'tatqa', 'techqa']:
# ragbench = load_dataset("rungalileo/ragbench", dataset)

In [ ]:
from datasets import concatenate_datasets

In [ ]:
dataset_train=concatenate_datasets([ragbench_covidqa, ragbench_hotpotqa,ragbench_cuad,ragbench_delucionqa,ragbench_emanual,ragbench_expertqa,ragbench_finqa,ragbench_hagrid,ragbench_msmarco,ragbench_pubmedqa,ragbench_tatqa,ragbench_techqa])

In [ ]:
dataset_train

Dataset({
    features: ['id', 'question', 'documents', 'response', 'generation_model_name', 'annotating_model_name', 'dataset_name', 'documents_sentences', 'response_sentences', 'sentence_support_information', 'unsupported_response_sentence_keys', 'adherence_score', 'overall_supported_explanation', 'relevance_explanation', 'all_relevant_sentence_keys', 'all_utilized_sentence_keys', 'trulens_groundedness', 'trulens_context_relevance', 'ragas_faithfulness', 'ragas_context_relevance', 'gpt3_adherence', 'gpt3_context_relevance', 'gpt35_utilization', 'relevance_score', 'utilization_score', 'completeness_score'],
    num_rows: 11802
})

In [ ]:
# Get all the train data into a single column

docs_train =[]
docs_train_qestions = []
docs_train_relevance = []
docs_train_utilization = []
docs_train_completeness = []
docs_train_ground_response = []
docs_train_adherance = []



In [ ]:
docs_train = dataset_train['documents']
#documents = [Document(page_content=doc[0]) for doc in docs_train]
documents = [Document(page_content=" ".join(doc)) for doc in docs_train if isinstance(doc, list)]

In [ ]:
docs_train_qestions = dataset_train['question']
#questions = [Document(page_content=doc[0]) for doc in docs_train_qestions]

docs_train_relevance = dataset_train['gpt3_context_relevance']

docs_train_utilization = dataset_train['utilization_score']

docs_train_completeness = dataset_train['completeness_score']

docs_train_ground_response = dataset_train['response']

docs_train_adherance = dataset_train['gpt3_adherence']



In [ ]:
len(docs_train_qestions)

11802

**Use the Sentence Splitter for Chunking**

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Paragraph-based chunking
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,  # Larger chunk size to fit multi-paragraph contexts
    chunk_overlap=100,  # Maintain overlapping for continuity
    separators=["\n\n", ".", "?", "!"],  # Added . ? ! as separators
)

In [ ]:
from langchain.docstore.document import Document

all_chunks = []
for doc in documents:  # `documents` is a list of Document objects
    if isinstance(doc, Document):
        chunks = text_splitter.split_text(doc.page_content)  # Use the `page_content` attribute
        all_chunks.extend(chunks)
    else:
        raise TypeError("Expected a Document object or string, but got: {}".format(type(doc)))

In [ ]:
all_chunks

['The Brooklyn Nets are an American professional basketball team based in Brooklyn, New York. They are a member of the Atlantic Division of the Eastern Conference in the National Basketball Association (NBA). The team plays its home games at the Barclays Center. The franchise was founded as the New Jersey Americans in 1967, and was one of the eleven original American Basketball Association (ABA) teams. In its second ABA season, Arthur Brown, the team owner, moved the team to Long Island and renamed it the New York Nets. The team won ABA championships in 1974 and 1976. When the ABA merged with the NBA in 1976, the Nets were one of four ABA teams admitted into the NBA',
 '. The team was moved to the Rutgers Athletic Center in New Jersey; after the 1976–77 NBA season, the team was renamed the New Jersey Nets. Since they joined the NBA, the Nets have won 4\xa0divisional championships, 2\xa0conference championships and appeared in the playoffs 16\xa0times. The Nets moved to Brooklyn in 2012

Step 3: Create embeddings and store in Vector Database (3 points)
Your are encouraged to use open source embedding models from mteb/leaderboard [Massive Text Embedding Benchmark (MTEB) Leaderboard](https://https://huggingface.co/spaces/mteb/leaderboard)

**For Embedding Use all-mpnet-base-v2 and for VectorDb , use Chroma**

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
#device = "cpu"

In [ ]:

model_name = "sentence-transformers/msmarco-bert-base-dot-v5"#"sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': device,'trust_remote_code': True }
encode_kwargs = {'normalize_embeddings': False}
embedding = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.19k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.vectorstores import Chroma
import os

In [ ]:
persist_directory = 'docs/chroma/GP'
if os.path.exists(persist_directory):
  !rm -rf ./docs/chroma/
else:
  os.makedirs(persist_directory, exist_ok=True)

In [ ]:
from langchain.docstore.document import Document

# Assuming 'all_chunks' contains the text chunks as strings
documents = [Document(page_content=chunk) for chunk in all_chunks]

In [ ]:
vectordb = Chroma.from_documents(
    documents=documents,#splits, # splits we created earlier
    embedding=embedding,
    persist_directory= persist_directory, # save the directory
)

In [ ]:
print(vectordb._collection.count())

1242


In [ ]:
import shutil
from google.colab import files

# Path to your persist directory (adjust as needed)
persist_directory = 'docs/chroma/GP'

# Create a zip file of the persist directory
shutil.make_archive("/content/persist_dir_GP", 'zip', persist_directory)



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Rank the Retrieved documents**

In [ ]:
# # Filter the retrieved documents to keep only the most relevant ones
# def filter_retrieved_docs(docs, top_n=5):
#     # Assign a default relevance score if missing
#     for doc in docs:
#         if 'relevance_score' not in doc.metadata:
#             doc.metadata['relevance_score'] = 0  # Assign a default score

#     # Sort documents by relevance score
#     sorted_docs = sorted(docs, key=lambda x: x.metadata['relevance_score'], reverse=True)
#     return sorted_docs[:top_n]

**Loop through the Question set and convert the output to DataFrame**

In [ ]:
# # Loop through each question and invoke the retriever
# docs_list = []
# retriever_final = vectordb.as_retriever(search_type="mmr",search_kwargs={"k": 5, "fetch_k":10})

# def get_retrieved_data(questions):
#     data = []
#     for question in questions:
#         try:
#             context = retriever_final.invoke(question)  # Retrieve context
#             data.append({"Question": question, "Retrieved Context": context})
#         except Exception as e:
#             data.append({"Question": question, "Retrieved Context": f"Error: {str(e)}"})
#     return data


In [ ]:
# import pandas as pd

In [ ]:
# # Process the questions
# retrieved_data = get_retrieved_data(docs_train_qestions)
# # Convert results into a DataFrame
# df = pd.DataFrame(retrieved_data)

In [ ]:
# df

,Question,Retrieved Context
0,Which university did one of the key figures in...,[page_content='. The team was moved to the Rut...
1,Were both Léopold Eyharts and Ulrich Walter a ...,"[page_content='Léopold Eyharts (born April 28,..."
2,"Which music group has the most members, DC Tal...",[page_content='DC Talk (stylized as dc Talk) i...
3,Eddie Jemison had a role in the television ser...,"[page_content='. Edward Francis ""Eddie"" Jemiso..."
4,Consider the band formed in 2015 by JYP Entert...,[page_content='. It was released by JYP Entert...
...,...,...
385,What actor had his final performance in the fi...,[page_content='The Circle is a 2017 American t...
386,Between Danny Elfman and Fran Healy who has wo...,[page_content='. Their early work was key to t...
387,Which one the cast members in the film Only Lo...,[page_content='. He won the Laurence Olivier A...
388,Considered the strongest recorded tropical cyc...,[page_content='. Interaction with the island c...


With Hyde

In [ ]:
# df["Grounded Response"] = docs_train_ground_response

Step 4: Perform the retrieval augmented generation by integrating with LLM (3 points)
Experiment with and without MMR and check the final response. It is encouraged to use open source[ LLMs](https://huggingface.co/models?other=text-generation-inference&sort=trending).

**Creating the RAG chain**

In [ ]:
# # Library to communicate with HF hub
# #!pip install huggingface_hub
# from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace

In [ ]:
# from langchain_huggingface import HuggingFacePipeline

In [ ]:
# #from langchain_community.chat_models import ChatOllama
# from langchain_core.output_parsers import StrOutputParser

In [ ]:
# from langchain.chains import RetrievalQA
# from langchain.prompts import PromptTemplate
# from langchain.llms import HuggingFacePipeline
# from transformers import pipeline

In [ ]:
# prompt = """
# Based on the following documents:
# {documents}

# Answer the question:
# {question}

# Your answer has to be based on the documents only.If the {question} is asking for steps, please answer in bullet points.If you do not know the answer, just say Unfortunately hte document doesnt contain relevant information.Do not make up an answer.Limit the answer between 1 to 3 sentences only and has to be on point.
# """


In [ ]:
# prompt = PromptTemplate(input_variables=["documents", "question"], template=prompt)

In [ ]:
# %pip install --upgrade --quiet huggingface_hub

In [ ]:
# from getpass import getpass

# HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [ ]:
# import os

# os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
# from langchain_huggingface import HuggingFaceEndpoint

In [ ]:
# from langchain.chains import LLMChain
# from langchain_core.prompts import PromptTemplate

In [ ]:
# repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

# llm = HuggingFaceEndpoint(
#     repo_id=repo_id,
#     max_length=128,
#     temperature=0.5,
#     huggingfacehub_api_token=HUGGINGFACEHUB_API_TOKEN,
# )



                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.


In [ ]:
# llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
# def process_row(row):
#     documents = row["Retrieved Context"]
#     question = row["Question"]
#     response = llm_chain.run({"documents": documents, "question": question, "answer":""})
#     return response

**Use Parellel Processing**

In [ ]:
# from concurrent.futures import ThreadPoolExecutor

In [ ]:
# # Parallel processing of rows
# def process_dataframe_in_parallel(df, num_workers=10):
#     with ThreadPoolExecutor(max_workers=num_workers) as executor:
#         responses = list(executor.map(process_row, [row for _, row in df.iterrows()]))
#     return responses

In [ ]:
# # Run the processing
# df["Responses"] = process_dataframe_in_parallel(df)

In [ ]:
# df

,Question,Retrieved Context,Grounded Response,Responses
0,Which university did one of the key figures in...,[page_content='. The team was moved to the Rut...,One of the key figures in the American documen...,"\nAnswer:\nJason Collins, a key figure in the ..."
1,Were both Léopold Eyharts and Ulrich Walter a ...,"[page_content='Léopold Eyharts (born April 28,...","No, only Léopold Eyharts was a General in the ...","\nAnNo, Léopold Eyharts is a Brigadier General..."
2,"Which music group has the most members, DC Tal...",[page_content='DC Talk (stylized as dc Talk) i...,"DC Talk has three members, while Manchester Or...",\nAnswer:\nDC Talk is a Christian rap and rock...
3,Eddie Jemison had a role in the television ser...,"[page_content='. Edward Francis ""Eddie"" Jemiso...","March 17, 2015",\nAnswer:\nThe CW premiered the television ser...
4,Consider the band formed in 2015 by JYP Entert...,[page_content='. It was released by JYP Entert...,The band formed in 2015 by JYP Entertainment t...,\nAnswer:\nThe reissue of Twice's third album ...
...,...,...,...,...
385,What actor had his final performance in the fi...,[page_content='The Circle is a 2017 American t...,Bill Paxton had his final performance in the f...,\nAnswer: Bill Paxton had his final performanc...
386,Between Danny Elfman and Fran Healy who has wo...,[page_content='. Their early work was key to t...,"Based on the provided context, Danny Elfman ha...",\nAnswer:\nDanny Elfman is known for composing...
387,Which one the cast members in the film Only Lo...,[page_content='. He won the Laurence Olivier A...,Tom Hiddleston,\nTom Hiddleston was the cast member in the fi...
388,Considered the strongest recorded tropical cyc...,[page_content='. Interaction with the island c...,The 1999 Odisha cyclone had a film made about ...,"\nAnswer:\nUnfortunately, the documents do not..."


**Get the Actual prompt from the Research paper and pass to RAGCHAIN**

In [ ]:
# prompt_template ="""
# Here are the documents, each of which is split into sentences. Alongside each
# sentence is associated key, such as ’0a.’ or ’0b.’ that you can use to refer
# to it:
# ‘‘‘
# {documents}
# ‘‘‘
# The question was:
# ‘‘‘
# {question}
# ‘‘‘
# Here is their response, split into sentences. Alongside each sentence is
# associated key, such as ’a.’ or ’b.’ that you can use to refer to it. Note
# that these keys are unique to the response, and are not related to the keys
# in the documents:
# ‘‘‘
# {answer}
# ‘‘‘
# You must respond with a JSON object matching this schema:
# ‘‘‘
# {{
# "relevance_explanation": string,
# "all_relevant_sentence_keys": [string],
# "overall_supported_explanation": string,
# "overall_supported": boolean,
# "sentence_support_information": [
# {{
# "response_sentence_key": string,
# "explanation": string,
# "supporting_sentence_keys": [string],
# "fully_supported": boolean
# }},
# ],
# "all_utilized_sentence_keys": [string]
# }}
# ‘‘‘
# The relevance_explanation field is a string explaining which documents
# contain useful information for answering the question. Provide a step-by-step
# breakdown of information provided in the documents and how it is useful for
# answering the question.
# The all_relevant_sentence_keys field is a list of all document sentences keys
# (e.g. ’0a’) that are revant to the question. Include every sentence that is
# useful and relevant to the question, even if it was not used in the response,
# or if only parts of the sentence are useful. Ignore the provided response when
# making this judgement and base your judgement solely on the provided documents
# and question. Omit sentences that, if removed from the document, would not
# impact someone’s ability to answer the question.
# The overall_supported_explanation field is a string explaining why the response
# *as a whole* is or is not supported by the documents. In this field, provide a
# step-by-step breakdown of the claims made in the response and the support (or
# lack thereof) for those claims in the documents. Begin by assessing each claim
# separately, one by one; don’t make any remarks about the response as a whole
# until you have assessed all the claims in isolation.
# The overall_supported field is a boolean indicating whether the response as a
# whole is supported by the documents. This value should reflect the conclusion
# you drew at the end of your step-by-step breakdown in overall_supported_explanation.
# In the sentence_support_information field, provide information about the support
# *for each sentence* in the response.
# The sentence_support_information field is a list of objects, one for each sentence
# in the response. Each object MUST have the following fields:
# - response_sentence_key: a string identifying the sentence in the response.
# This key is the same as the one used in the response above.
# - explanation: a string explaining why the sentence is or is not supported by the
# documents.
# - supporting_sentence_keys: keys (e.g. ’0a’) of sentences from the documents that
# support the response sentence. If the sentence is not supported, this list MUST
# be empty. If the sentence is supported, this list MUST contain one or more keys.
# In special cases where the sentence is supported, but not by any specific sentence,
# you can use the string "supported_without_sentence" to indicate that the sentence
# is generally supported by the documents. Consider cases where the sentence is
# expressing inability to answer the question due to lack of relevant information in
# the provided contex as "supported_without_sentence". In cases where the sentence
# is making a general statement (e.g. outlining the steps to produce an answer, or
# summarizing previously stated sentences, or a transition sentence), use the
# sting "general".In cases where the sentence is correctly stating a well-known fact,
# like a mathematical formula, use the string "well_known_fact". In cases where the
# sentence is performing numerical reasoning (e.g. addition, multiplication), use
# the string "numerical_reasoning".
# - fully_supported: a boolean indicating whether the sentence is fully supported by
# the documents.
# - This value should reflect the conclusion you drew at the end of your step-by-step
# breakdown in explanation.
# - If supporting_sentence_keys is an empty list, then fully_supported must be false.
# - Otherwise, use fully_supported to clarify whether everything in the response
# sentence is fully supported by the document text indicated in supporting_sentence_keys
# (fully_supported = true), or whether the sentence is only partially or incompletely
# supported by that document text (fully_supported = false).
# The all_utilized_sentence_keys field is a list of all sentences keys (e.g. ’0a’) that
# were used to construct the answer. Include every sentence that either directly supported
# the answer, or was implicitly used to construct the answer, even if it was not used
# in its entirety. Omit sentences that were not used, and could have been removed from
# the documents without affecting the answer.
# You must respond with a valid JSON string. Use escapes for quotes, e.g. ‘\\"‘, and
# newlines, e.g. ‘\\n‘. Do not write anything before or after the JSON string. Do not
# wrap the JSON string in backticks like ‘‘‘ or ‘‘‘json.
# As a reminder: your task is to review the response and assess which documents contain
# useful information pertaining to the question, and how each sentence in the response
# is supported by the text in the documents.
#     """


In [ ]:
# prompt = PromptTemplate(input_variables=["documents", "question"], template=prompt_template)

In [ ]:
# llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
# response = process_dataframe_in_parallel(df)

In [ ]:
# df["Response from llm"] = response

In [ ]:
# df

,Question,Retrieved Context,Grounded Response,Responses,Response from llm
0,Which university did one of the key figures in...,[page_content='. The team was moved to the Rut...,One of the key figures in the American documen...,"\nAnswer:\nJason Collins, a key figure in the ...","\n‘‘‘\n{\n""relevance_explanation"": ""The docume..."
1,Were both Léopold Eyharts and Ulrich Walter a ...,"[page_content='Léopold Eyharts (born April 28,...","No, only Léopold Eyharts was a General in the ...","\nAnNo, Léopold Eyharts is a Brigadier General...","\n‘‘‘\n{\n""relevance_explanation"": ""The docume..."
2,"Which music group has the most members, DC Tal...",[page_content='DC Talk (stylized as dc Talk) i...,"DC Talk has three members, while Manchester Or...",\nAnswer:\nDC Talk is a Christian rap and rock...,"\n‘‘‘\nHere is the response:\n{\n""relevance_ex..."
3,Eddie Jemison had a role in the television ser...,"[page_content='. Edward Francis ""Eddie"" Jemiso...","March 17, 2015",\nAnswer:\nThe CW premiered the television ser...,"\n‘‘‘\nResponse:\n{\n""relevance_explanation"": ..."
4,Consider the band formed in 2015 by JYP Entert...,[page_content='. It was released by JYP Entert...,The band formed in 2015 by JYP Entertainment t...,\nAnswer:\nThe reissue of Twice's third album ...,"\n‘‘‘\nResponse:\n{\n""relevance_explanation"": ..."
...,...,...,...,...,...
385,What actor had his final performance in the fi...,[page_content='The Circle is a 2017 American t...,Bill Paxton had his final performance in the f...,\nAnswer: Bill Paxton had his final performanc...,\n‘‘‘\nThe actor who had his final performance...
386,Between Danny Elfman and Fran Healy who has wo...,[page_content='. Their early work was key to t...,"Based on the provided context, Danny Elfman ha...",\nAnswer:\nDanny Elfman is known for composing...,"\n‘‘‘\nHere is the response:\n‘‘‘\n{\n""relevan..."
387,Which one the cast members in the film Only Lo...,[page_content='. He won the Laurence Olivier A...,Tom Hiddleston,\nTom Hiddleston was the cast member in the fi...,"\nHere is the response to assess:\n[\n{\n""rele..."
388,Considered the strongest recorded tropical cyc...,[page_content='. Interaction with the island c...,The 1999 Odisha cyclone had a film made about ...,"\nAnswer:\nUnfortunately, the documents do not...","\n‘‘‘\nThe response was:\n‘‘‘\n{\n""relevance_e..."


**Extract JSON response and pretty the output**

In [ ]:
from sklearn.metrics import mean_squared_error, roc_auc_score

In [ ]:
import pandas as pd
import json

In [ ]:
# def extract_json_from_llm_response(response):
#     try:
#         # Check if response is empty or None
#         if not response or not isinstance(response, str):
#             raise ValueError("Input response is empty or not a valid string.")

#         # Find the JSON substring
#         start = response.find("{")
#         end = response.rfind("}") + 1

#         # Ensure valid JSON-like structure is found
#         if start == -1 or end == 0:
#             raise ValueError("No valid JSON object found in the response.")

#         json_string = response[start:end]

#         # Parse the JSON string
#         extracted_json = json.loads(json_string)

#         # Format the JSON for pretty printing
#         formatted_json = json.dumps(extracted_json, indent=4)
#         return formatted_json
#     except (json.JSONDecodeError, ValueError) as e:
#         print(f"Error processing response: '{response}'. Error: {e}")
#         return None

In [ ]:
# # Apply the function to the DataFrame column
# df['formatted_json'] = df['Response from llm'].apply(extract_json_from_llm_response)

Error processing response: '
‘‘‘
{
"relevance_explanation": "The document about the American documentary film mentions Jason Collins, who is a basketball player and a key figure in the film. The document also mentions that Jason Collins played for the New Jersey Nets. However, it does not mention which university he attended before being drafted by the New Jersey Nets.",
"all_relevant_sentence_keys": ["0a"],
"overall_supported_explanation": "The response correctly identifies Jason Collins as a basketball player and a key figure in the American documentary film. It also correctly identifies that Jason Collins played for the New Jersey Nets. However, the response goes on to incorrectly claim that Jason Collins attended a university before being drafted by the New Jersey Nets, but the document does not provide any information about which university that might be. Therefore, the response is only partially supported by the documents, as it is only correct up to the point where it claims Jas

In [ ]:
# df

,Question,Retrieved Context,Grounded Response,Responses,Response from llm,formatted_json
0,Which university did one of the key figures in...,[page_content='. The team was moved to the Rut...,One of the key figures in the American documen...,"\nAnswer:\nJason Collins, a key figure in the ...","\n‘‘‘\n{\n""relevance_explanation"": ""The docume...",None
1,Were both Léopold Eyharts and Ulrich Walter a ...,"[page_content='Léopold Eyharts (born April 28,...","No, only Léopold Eyharts was a General in the ...","\nAnNo, Léopold Eyharts is a Brigadier General...","\n‘‘‘\n{\n""relevance_explanation"": ""The docume...","{\n ""relevance_explanation"": ""The documents..."
2,"Which music group has the most members, DC Tal...",[page_content='DC Talk (stylized as dc Talk) i...,"DC Talk has three members, while Manchester Or...",\nAnswer:\nDC Talk is a Christian rap and rock...,"\n‘‘‘\nHere is the response:\n{\n""relevance_ex...","{\n ""relevance_explanation"": ""The documents..."
3,Eddie Jemison had a role in the television ser...,"[page_content='. Edward Francis ""Eddie"" Jemiso...","March 17, 2015",\nAnswer:\nThe CW premiered the television ser...,"\n‘‘‘\nResponse:\n{\n""relevance_explanation"": ...","{\n ""relevance_explanation"": ""The documents..."
4,Consider the band formed in 2015 by JYP Entert...,[page_content='. It was released by JYP Entert...,The band formed in 2015 by JYP Entertainment t...,\nAnswer:\nThe reissue of Twice's third album ...,"\n‘‘‘\nResponse:\n{\n""relevance_explanation"": ...",None
...,...,...,...,...,...,...
385,What actor had his final performance in the fi...,[page_content='The Circle is a 2017 American t...,Bill Paxton had his final performance in the f...,\nAnswer: Bill Paxton had his final performanc...,\n‘‘‘\nThe actor who had his final performance...,None
386,Between Danny Elfman and Fran Healy who has wo...,[page_content='. Their early work was key to t...,"Based on the provided context, Danny Elfman ha...",\nAnswer:\nDanny Elfman is known for composing...,"\n‘‘‘\nHere is the response:\n‘‘‘\n{\n""relevan...","{\n ""relevance_explanation"": ""The documents..."
387,Which one the cast members in the film Only Lo...,[page_content='. He won the Laurence Olivier A...,Tom Hiddleston,\nTom Hiddleston was the cast member in the fi...,"\nHere is the response to assess:\n[\n{\n""rele...","{\n ""relevance_explanation"": ""The documents..."
388,Considered the strongest recorded tropical cyc...,[page_content='. Interaction with the island c...,The 1999 Odisha cyclone had a film made about ...,"\nAnswer:\nUnfortunately, the documents do not...","\n‘‘‘\nThe response was:\n‘‘‘\n{\n""relevance_e...",None


In [ ]:
# # Convert JSON strings to Python dictionaries
# df['attributes'] = df['formatted_json'].apply(lambda x: json.loads(x) if x else None)


In [ ]:
# df

,Question,Retrieved Context,Grounded Response,Responses,Response from llm,formatted_json,attributes
0,Which university did one of the key figures in...,[page_content='. The team was moved to the Rut...,One of the key figures in the American documen...,"\nAnswer:\nJason Collins, a key figure in the ...","\n‘‘‘\n{\n""relevance_explanation"": ""The docume...",None,None
1,Were both Léopold Eyharts and Ulrich Walter a ...,"[page_content='Léopold Eyharts (born April 28,...","No, only Léopold Eyharts was a General in the ...","\nAnNo, Léopold Eyharts is a Brigadier General...","\n‘‘‘\n{\n""relevance_explanation"": ""The docume...","{\n ""relevance_explanation"": ""The documents...",{'relevance_explanation': 'The documents provi...
2,"Which music group has the most members, DC Tal...",[page_content='DC Talk (stylized as dc Talk) i...,"DC Talk has three members, while Manchester Or...",\nAnswer:\nDC Talk is a Christian rap and rock...,"\n‘‘‘\nHere is the response:\n{\n""relevance_ex...","{\n ""relevance_explanation"": ""The documents...",{'relevance_explanation': 'The documents provi...
3,Eddie Jemison had a role in the television ser...,"[page_content='. Edward Francis ""Eddie"" Jemiso...","March 17, 2015",\nAnswer:\nThe CW premiered the television ser...,"\n‘‘‘\nResponse:\n{\n""relevance_explanation"": ...","{\n ""relevance_explanation"": ""The documents...",{'relevance_explanation': 'The documents provi...
4,Consider the band formed in 2015 by JYP Entert...,[page_content='. It was released by JYP Entert...,The band formed in 2015 by JYP Entertainment t...,\nAnswer:\nThe reissue of Twice's third album ...,"\n‘‘‘\nResponse:\n{\n""relevance_explanation"": ...",None,None
...,...,...,...,...,...,...,...
385,What actor had his final performance in the fi...,[page_content='The Circle is a 2017 American t...,Bill Paxton had his final performance in the f...,\nAnswer: Bill Paxton had his final performanc...,\n‘‘‘\nThe actor who had his final performance...,None,None
386,Between Danny Elfman and Fran Healy who has wo...,[page_content='. Their early work was key to t...,"Based on the provided context, Danny Elfman ha...",\nAnswer:\nDanny Elfman is known for composing...,"\n‘‘‘\nHere is the response:\n‘‘‘\n{\n""relevan...","{\n ""relevance_explanation"": ""The documents...",{'relevance_explanation': 'The documents menti...
387,Which one the cast members in the film Only Lo...,[page_content='. He won the Laurence Olivier A...,Tom Hiddleston,\nTom Hiddleston was the cast member in the fi...,"\nHere is the response to assess:\n[\n{\n""rele...","{\n ""relevance_explanation"": ""The documents...",{'relevance_explanation': 'The documents conta...
388,Considered the strongest recorded tropical cyc...,[page_content='. Interaction with the island c...,The 1999 Odisha cyclone had a film made about ...,"\nAnswer:\nUnfortunately, the documents do not...","\n‘‘‘\nThe response was:\n‘‘‘\n{\n""relevance_e...",None,None


**Evaluate the metrics**

In [ ]:
# def compute_metrics(attributes, total_doc_sentences):
#     # Use .get() to avoid KeyError
#     relevant_keys = set(attributes.get("all_relevant_sentence_keys", []))
#     utilized_keys = set(attributes.get("all_utilized_sentence_keys", []))
#     sentence_support_info = attributes.get("sentence_support_information", [])
#     supported_sentences = [info for info in sentence_support_info if info.get("fully_supported", False)]
#     total_response_sentences = len(sentence_support_info)
#     overall_supported = attributes.get("overall_supported", False)

#     context_relevance = len(relevant_keys) / total_doc_sentences if total_doc_sentences > 0 else 0
#     context_utilization = len(utilized_keys) / len(relevant_keys) if relevant_keys else 0
#     completeness = len(supported_sentences) / total_response_sentences if total_response_sentences > 0 else 0
#     adherence = 1 if overall_supported else 0
#     return {
#         "Context Relevance": context_relevance,
#         "Context Utilization": context_utilization,
#         "Completeness": completeness,
#         "Adherence": adherence
#     }

In [ ]:
# def calculate_row_metrics(row):
#     # Debugging information
#     if row.get("Retrieved Context") is None or row.get("attributes") is None:
#         print(f"Skipping row due to missing data: {row}")
#         return None

#     # Continue with the rest of the function
#     if isinstance(row["Retrieved Context"], list):
#         document_text = "\n".join(
#             str(doc.content if hasattr(doc, "content") else doc) for doc in row["Retrieved Context"]
#         )
#     else:
#         document_text = row["Retrieved Context"]

#     document_text = str(document_text)
#     total_doc_sentences = len(document_text.split("\n"))

#     return compute_metrics(row["attributes"], total_doc_sentences)


In [ ]:
# metrics_df = df.apply(lambda row: calculate_row_metrics(row), axis=1)

Skipping row due to missing data: Question             Which university did one of the key figures in...
Retrieved Context    [page_content='. The team was moved to the Rut...
Grounded Response    One of the key figures in the American documen...
Responses            \nAnswer:\nJason Collins, a key figure in the ...
Response from llm    \n‘‘‘\n{\n"relevance_explanation": "The docume...
formatted_json                                                    None
attributes                                                        None
Name: 0, dtype: object
Skipping row due to missing data: Question             Consider the band formed in 2015 by JYP Entert...
Retrieved Context    [page_content='. It was released by JYP Entert...
Grounded Response    The band formed in 2015 by JYP Entertainment t...
Responses            \nAnswer:\nThe reissue of Twice's third album ...
Response from llm    \n‘‘‘\nResponse:\n{\n"relevance_explanation": ...
formatted_json                                           

In [ ]:
# # Add new columns to the DataFrame
# df["Context Relevance"] = metrics_df.apply(lambda x: x["Context Relevance"] if x is not None else None)
# df["Context Utilization"] = metrics_df.apply(lambda x: x["Context Utilization"] if x is not None else None)
# df["Completeness"] = metrics_df.apply(lambda x: x["Completeness"] if x is not None else None)
# df["Adherence"] = metrics_df.apply(lambda x: x["Adherence"] if x is not None else None)


In [ ]:
# df

,Question,Retrieved Context,Grounded Response,Responses,Response from llm,formatted_json,attributes,Context Relevance,Context Utilization,Completeness,Adherence
0,Which university did one of the key figures in...,[page_content='. The team was moved to the Rut...,One of the key figures in the American documen...,"\nAnswer:\nJason Collins, a key figure in the ...","\n‘‘‘\n{\n""relevance_explanation"": ""The docume...",None,None,NaN,NaN,NaN,NaN
1,Were both Léopold Eyharts and Ulrich Walter a ...,"[page_content='Léopold Eyharts (born April 28,...","No, only Léopold Eyharts was a General in the ...","\nAnNo, Léopold Eyharts is a Brigadier General...","\n‘‘‘\n{\n""relevance_explanation"": ""The docume...","{\n ""relevance_explanation"": ""The documents...",{'relevance_explanation': 'The documents provi...,0.2,1.0,0.000000,0.0
2,"Which music group has the most members, DC Tal...",[page_content='DC Talk (stylized as dc Talk) i...,"DC Talk has three members, while Manchester Or...",\nAnswer:\nDC Talk is a Christian rap and rock...,"\n‘‘‘\nHere is the response:\n{\n""relevance_ex...","{\n ""relevance_explanation"": ""The documents...",{'relevance_explanation': 'The documents provi...,0.0,0.0,0.750000,0.0
3,Eddie Jemison had a role in the television ser...,"[page_content='. Edward Francis ""Eddie"" Jemiso...","March 17, 2015",\nAnswer:\nThe CW premiered the television ser...,"\n‘‘‘\nResponse:\n{\n""relevance_explanation"": ...","{\n ""relevance_explanation"": ""The documents...",{'relevance_explanation': 'The documents provi...,0.4,1.0,1.000000,1.0
4,Consider the band formed in 2015 by JYP Entert...,[page_content='. It was released by JYP Entert...,The band formed in 2015 by JYP Entertainment t...,\nAnswer:\nThe reissue of Twice's third album ...,"\n‘‘‘\nResponse:\n{\n""relevance_explanation"": ...",None,None,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
385,What actor had his final performance in the fi...,[page_content='The Circle is a 2017 American t...,Bill Paxton had his final performance in the f...,\nAnswer: Bill Paxton had his final performanc...,\n‘‘‘\nThe actor who had his final performance...,None,None,NaN,NaN,NaN,NaN
386,Between Danny Elfman and Fran Healy who has wo...,[page_content='. Their early work was key to t...,"Based on the provided context, Danny Elfman ha...",\nAnswer:\nDanny Elfman is known for composing...,"\n‘‘‘\nHere is the response:\n‘‘‘\n{\n""relevan...","{\n ""relevance_explanation"": ""The documents...",{'relevance_explanation': 'The documents menti...,0.4,1.0,0.333333,0.0
387,Which one the cast members in the film Only Lo...,[page_content='. He won the Laurence Olivier A...,Tom Hiddleston,\nTom Hiddleston was the cast member in the fi...,"\nHere is the response to assess:\n[\n{\n""rele...","{\n ""relevance_explanation"": ""The documents...",{'relevance_explanation': 'The documents conta...,0.4,1.0,0.500000,0.0
388,Considered the strongest recorded tropical cyc...,[page_content='. Interaction with the island c...,The 1999 Odisha cyclone had a film made about ...,"\nAnswer:\nUnfortunately, the documents do not...","\n‘‘‘\nThe response was:\n‘‘‘\n{\n""relevance_e...",None,None,NaN,NaN,NaN,NaN


**Ooutput the Dataframe to xls**

In [ ]:
# df.to_excel('LLM_Response.xlsx', index=False)

**Still need to work on **

In [ ]:
# import numpy as np
import numpy as np
from sklearn.metrics import mean_squared_error, roc_auc_score
import pandas as pd

In [ ]:
# df["grounded_context_relevance"] = docs_train_relevance
# df["grounded_context_utilization"] = docs_train_utilization
# df["grounded_adherence"] = docs_train_adherance

In [ ]:
# df = df.dropna()

In [ ]:
# # RMSE Calculation for each metric
# rmse_context_relevance = np.sqrt(mean_squared_error(df["grounded_context_relevance"], df["Context Relevance"]))
# rmse_context_utilization = np.sqrt(mean_squared_error(df["grounded_context_utilization"], df["Context Utilization"]))
# rmse_adherence = np.sqrt(mean_squared_error(df["grounded_adherence"], df["Adherence"]))

# # AUC-ROC Calculation for each metric (only binary data for adherence)
# auc_context_relevance = roc_auc_score((df["grounded_context_relevance"] > 0.5).astype(int), df["Context Relevance"])
# auc_context_utilization = roc_auc_score((df["grounded_context_utilization"] > 0.5).astype(int), df["Context Utilization"])
# auc_adherence = roc_auc_score(df["grounded_adherence"], df["Adherence"])

# # Display Results
# print(f"RMSE for Context Relevance: {rmse_context_relevance:.4f}")
# print(f"RMSE for Context Utilization: {rmse_context_utilization:.4f}")
# print(f"RMSE for Adherence: {rmse_adherence:.4f}")

# print(f"AUC-ROC for Context Relevance: {auc_context_relevance:.4f}")
# print(f"AUC-ROC for Context Utilization: {auc_context_utilization:.4f}")
# print(f"AUC-ROC for Adherence: {auc_adherence:.4f}")

RMSE for Context Relevance: 0.2185
RMSE for Context Utilization: 0.5192
RMSE for Adherence: 0.8428
AUC-ROC for Context Relevance: 0.5702
AUC-ROC for Context Utilization: nan
AUC-ROC for Adherence: 0.5515


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_ranking.py:375: UndefinedMetricWarning: Only one class is present in y_true. ROC AUC score is not defined in that case.
  warnings.warn(


In [ ]:
# def compute_auc_roc(predicted, ground_truth):
#     try:
#         auc = roc_auc_score(ground_truth, predicted)
#         return auc
#     except ValueError:

#         return None  # Return None if AUC cannot be computed due to data issues

In [ ]:
# predicted_values = [
#         computed_metrics["Context Relevance"],
#         computed_metrics["Context Utilization"],
#         computed_metrics["Completeness"],
#     ]
# ground_truth_metrics = [0.3, 0.2, 1.0]

# rmse = evaluate_metrics(predicted_values, ground_truth_metrics)
# print("RMSE:",rmse)

In [ ]:
# ground_truth_adherence = [1]

In [ ]:
# to compute AUCROC, we will need to pass the entire question set to llm and get different result
# adherence_auc = compute_auc_roc([computed_metrics["Adherence"]],[ground_truth_adherence])

In [ ]:
# print(adherence_auc)

Step 7: Deploy the application on HF Spaces [Optional]